In [13]:
%pip install git+https://github.com/huggingface/transformers
%pip install langchain chromadb pypdf openai sentence-transformers accelerate langchain-community python-docx

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\lokesh\AppData\Local\Temp\pip-req-build-pm_ekyy_'

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Cloning https://github.com/huggingface/transformers to c:\users\lokesh\appdata\local\temp\pip-req-build-pm_ekyy_
  Resolved https://github.com/huggingface/transformers to commit 048f599f3506e57e0a595b455d9d2834c8d45023
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain in c:\python311\lib\site-packages (0.2.6)




[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

# model_kwargs = {'device': 'cuda'}
# embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True,)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300, temperature = 0.01)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
import pandas as pd
import pathlib
import docx
from langchain.docstore.document import Document
import os

def read_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([paragraph.text for paragraph in doc.paragraphs])

def load_all_files(directory_path):
    data = []
    for file_path in pathlib.Path(directory_path).glob("*"):
        if file_path.suffix == '.csv':
            df = pd.read_csv(file_path)
            for _, row in df.iterrows():
                content = " ".join(str(value) for value in row.values)
                data.append(Document(page_content=content))
        elif file_path.suffix == '.txt':
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                data.append(Document(page_content=content))
        elif file_path.suffix == '.docx':
            content = read_docx(file_path)
            data.append(Document(page_content=content))
        elif file_path.suffix == '.xlsx':
            df = pd.read_excel(file_path)
            for _, row in df.iterrows():
                content = " ".join(str(value) for value in row.values)
                data.append(Document(page_content=content))
    return data


In [ ]:
documents = load_all_files('D:\Axis-FAQ-chatbot\Data')
# documents


In [ ]:
# Load the PDF file
# pdf_link = "/content/Technical Design Document - Axis (2).pdf"
# loader = PyPDFLoader(pdf_link, extract_images=False)
# pages = loader.load_and_split()


# Split data into chunks
text_splitter = RecursiveCharacterTextSplitter()
chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
# Store data into database
db=Chroma.from_documents(chunks,embedding=embeddings,persist_directory="test_index")
db.persist()

In [ ]:
# Load the database
vectordb = Chroma(persist_directory="test_index", embedding_function = embeddings)

# Load the retriver
retriever = vectordb.as_retriever(search_kwargs = {"k" : 3})

In [ ]:
qna_prompt_template= """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Answer:"""

PROMPT = PromptTemplate(
   template=qna_prompt_template, input_variables=["context","question"] # Change 'Context' to 'context' and 'Question' to 'question'
)

# Define the QNA chain
chain = RetrievalQA.from_chain_type(llm=llm,
                                         chain_type='stuff',
                                         retriever=retriever,
                                         return_source_documents=True,
                                         chain_type_kwargs={'prompt': PROMPT})
#chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT,retriever=retriever)

In [ ]:
def answer_question(question):
    output = chain({"query": question})
    response = output['result']

    # Debugging: Print the raw response
    # print("Raw response:", response)

    # Post-process to remove the extraneous information
    if "Answer:" in response:
        response = response.split("Answer:")[1].strip()

    # Further cleanup if necessary
    response_lines = response.split('\n')
    cleaned_response = ""
    for line in response_lines:
        if not line.startswith("Question:") and not line.startswith("Answer:"):
            cleaned_response += line.strip() + " "

    return cleaned_response.strip()

In [12]:
question = input("Please enter your question: ")
answer = answer_question(question)

print(f"Answer: {answer}")

KeyboardInterrupt: 